# Dichotomic Pattern Mining (DPM)

In [1]:
import pandas as pd
from ast import literal_eval
from time import time
from IPython.display import display

from sequential.seq2pat import Seq2Pat, Attribute
from sequential.pat2feat import Pat2Feat
from sequential.dpm import dichotomic_pattern_mining, DichotomicAggregation

## Arguments

In [2]:
args = {}
args['data'] = '../tests/data/sample_data.csv'
args['min_frequency_pos'] = 0.3
args['min_frequency_neg'] = 0.3
args['max_span'] = 10

## Sample Data
- This notebook is going to run DPM on a sample sequences dataset, which is extracted from the published dataset in E-commerce Shopper Intent Prediction (Requena et al., 2020). The sequences are associated with positive or negative labels, e.g. purchase vs. non-purchase.

In [3]:
sequence_df = pd.read_csv(args['data'])
sequence_df.head()

,event_sequence,event_time,label
0,"[1, 1, 1, 2, 3, 1, 4, 1, 2, 3, 1, 4, 1, 2, 1, ...","[13118, 17085, 11839, 41749, 35195, 3348, 3309...",1
1,"[2, 1, 2, 1, 2, 1, 2, 1, 2, 3, 1, 2, 3, 1, 2, ...","[50205, 32403, 51377, 4256, 52139, 15020, 6999...",1
2,"[1, 1, 1, 2, 1, 2, 1, 1, 1, 1]","[49647, 45922, 26113, 422659, 9128, 82561, 709...",1
3,"[1, 1, 2, 1, 2, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, ...","[355031, 50126, 26262, 44512, 39795, 49730, 14...",1
4,"[1, 2, 1, 1, 1]","[19173, 159782, 12811, 88544, 53858]",1


### Transform sequence from string to list

In [4]:
literal_columns = ['event_sequence', 'event_time']

for column in literal_columns:
    sequence_df[column] = sequence_df[column].apply(literal_eval)

# Input lists
sequences = sequence_df['event_sequence'].values.tolist()
times = sequence_df['event_time'].values.tolist()

## Data Exploration

In [5]:
# EDA for items, max length, average length, number of positive and negative
num_sequences = len(sequence_df)
max_len = sequence_df['event_sequence'].apply(len).max()
avg_len = sequence_df['event_sequence'].apply(len).mean()
num_pos = len(sequence_df[sequence_df['label']==1])

print(f'Number of sequences: {num_sequences}')
print(f'Maximum length: {max_len}')
print(f'Average length: {avg_len}')
print(f'Number of positives: {num_pos}; Number of negatives: {num_sequences - num_pos}')

Number of sequences: 2000
Maximum length: 155
Average length: 28.3755
Number of positives: 1000; Number of negatives: 1000


## Seq2Pat for Positive Labels
- There is one attribute for each event: `event_time`
- Constraint 1: This constraint is to enforce the average event time greater than 20 sec
- Constraint 2: The built-in constraint in Seq2Pat, which can be configured by max_span parameter. This is to enforce the pattern mining to be within a span of `max_span` items (max_span=10 by default). 

In [6]:
# Get sequences having positive labels, and associated attributes.
sequences_pos = sequence_df[sequence_df['label']==1]['event_sequence'].values.tolist()
times_pos = sequence_df[sequence_df['label']==1]['event_time'].values.tolist()

seq2pat_pos = Seq2Pat(sequences_pos)

# Define a constraint on event time, average time >= 20 sec
time_attr_pos = Attribute(times_pos)
time_ct_pos = 20000 <= time_attr_pos.average()

# Add constraints to seq2pat
cs_pos = seq2pat_pos.add_constraint(time_ct_pos)

## Seq2Pat for Negative Labels
- Here we apply the same constraint models to sequences with negative labels

In [7]:
# Get sequences having positive labels, and associated attributes.
sequences_neg = sequence_df[sequence_df['label']==0]['event_sequence'].values.tolist()
times_neg = sequence_df[sequence_df['label']==0]['event_time'].values.tolist()

seq2pat_neg = Seq2Pat(sequences_neg)

# Define a constraint on event time, average time >= 20 sec
time_attr_neg = Attribute(times_neg)
time_ct_neg = 20000 <= time_attr_neg.average()

# Add constraints to seq2pat
cs_neg = seq2pat_neg.add_constraint(time_ct_neg)

## Dichotomic Pattern Mining: From Sequences to Patterns
- In DPM, we utilize the two Seq2Pat models for positive and negative sequences, mine the patterns that are frequent in each outcome and return different aggregations of mined patterns from the two cohorts.

In [8]:
t = time()

# Run DPM on positive and negative patterns and return a dict of pattern aggregations
aggregation_to_patterns = dichotomic_pattern_mining(seq2pat_pos, seq2pat_neg,
                                                    args['min_frequency_pos'],
                                                    args['min_frequency_neg'])

print(f'DPM finished! Runtime: {time()-t:.4f} sec')
for aggregation, patterns in aggregation_to_patterns.items():
    print("Aggregation: ", aggregation, " with number of patterns: ", len(patterns))

DPM finished! Runtime: 7.4062 sec
Aggregation:  intersection  with number of patterns:  215
Aggregation:  union  with number of patterns:  498
Aggregation:  unique_negative  with number of patterns:  29
Aggregation:  unique_positive  with number of patterns:  254


## From Patterns to Encodings
- Finally, we generate encodings of all sequences based on an aggregation of patterns found by DPM.

In [9]:
# Notice that constraints are optional to the generation of encodings
# In the following, we define a constraint on event time for all sequences, average time >= 20 sec
# The Seq2Pat built-in span constraint can be enforced in encodings generation by setting `max_span=10`.
time_attr = Attribute(times)
time_ct = 20000 <= time_attr.average()

# List of constraints 
constraints = [time_ct]

for aggregation, patterns in aggregation_to_patterns.items():
    print("Aggregation: ", aggregation)
    
    t = time()
    # find one hot encoding of each sequence for each pattern subject to constraints
    pat2feat = Pat2Feat()
    encodings = pat2feat.get_features(sequences, patterns, constraints, args['max_span'],
                                      drop_pattern_frequency=False)
    
    print(f'Encoding finished! Runtime: {time()-t:.4f} sec')
    display(encodings.head())

Aggregation:  intersection
Encoding finished! Runtime: 210.1841 sec


,sequence,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_205,feature_206,feature_207,feature_208,feature_209,feature_210,feature_211,feature_212,feature_213,feature_214
0,"[1, 1, 1, 2, 3, 1, 4, 1, 2, 3, 1, 4, 1, 2, 1, ...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[2, 1, 2, 1, 2, 1, 2, 1, 2, 3, 1, 2, 3, 1, 2, ...",1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,"[1, 1, 1, 2, 1, 2, 1, 1, 1, 1]",1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[1, 1, 2, 1, 2, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, ...",1,1,1,1,1,1,0,0,0,...,1,1,0,0,0,0,0,0,0,0
4,"[1, 2, 1, 1, 1]",1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Aggregation:  union
Encoding finished! Runtime: 405.0462 sec


,sequence,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_488,feature_489,feature_490,feature_491,feature_492,feature_493,feature_494,feature_495,feature_496,feature_497
0,"[1, 1, 1, 2, 3, 1, 4, 1, 2, 3, 1, 4, 1, 2, 1, ...",1,0,0,0,0,0,0,0,0,...,1,0,0,1,1,1,0,0,0,0
1,"[2, 1, 2, 1, 2, 1, 2, 1, 2, 3, 1, 2, 3, 1, 2, ...",1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1
2,"[1, 1, 1, 2, 1, 2, 1, 1, 1, 1]",1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[1, 1, 2, 1, 2, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, ...",1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"[1, 2, 1, 1, 1]",1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Aggregation:  unique_negative
Encoding finished! Runtime: 17.6183 sec


,sequence,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28
0,"[1, 1, 1, 2, 3, 1, 4, 1, 2, 3, 1, 4, 1, 2, 1, ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[2, 1, 2, 1, 2, 1, 2, 1, 2, 3, 1, 2, 3, 1, 2, ...",1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,"[1, 1, 1, 2, 1, 2, 1, 1, 1, 1]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[1, 1, 2, 1, 2, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, ...",0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,"[1, 2, 1, 1, 1]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Aggregation:  unique_positive
Encoding finished! Runtime: 186.6030 sec


,sequence,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_244,feature_245,feature_246,feature_247,feature_248,feature_249,feature_250,feature_251,feature_252,feature_253
0,"[1, 1, 1, 2, 3, 1, 4, 1, 2, 3, 1, 4, 1, 2, 1, ...",0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,1,0,0,0,0
1,"[2, 1, 2, 1, 2, 1, 2, 1, 2, 3, 1, 2, 3, 1, 2, ...",1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1
2,"[1, 1, 1, 2, 1, 2, 1, 1, 1, 1]",1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[1, 1, 2, 1, 2, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, ...",1,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,"[1, 2, 1, 1, 1]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
